# <font color='red'>**Libraries**</font>

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

In [ ]:
import tensorflow as tf
#import tensorflow_datasets as tfds
from tensorflow_examples.models.pix2pix import pix2pix
from os import listdir
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from numpy import vstack
from numpy import asarray
from numpy import savez_compressed
import numpy as np
from PIL import Image
from tqdm import tqdm
import os
import csv
import time
import matplotlib.pyplot as plt
from IPython.display import clear_output
from tensorflow import keras
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix, ConfusionMatrixDisplay
import imageio
from tensorflow import keras
from skimage.transform import resize

AUTOTUNE = tf.data.AUTOTUNE

# <font color='red'>**Useful methods**</font>

In [ ]:
def load_model(name):
    print("working on model: ", name)
    
    path = '../models/clasification/' + name + '.h5'
    
    model = keras.models.load_model(path, compile=False)
    
    return model

In [ ]:
def make_generator(current_df, HEIGHT, WIDTH, batch_size):
    
    test_datagen = ImageDataGenerator()

    test_generator=test_datagen.flow_from_dataframe(dataframe=current_df,
                                                    x_col="path",
                                                    y_col="label",
                                                    batch_size=batch_size,
                                                    seed=42,
                                                    shuffle=False,
                                                    class_mode="categorical",
                                                    target_size=(HEIGHT, WIDTH))
    
    return test_generator

# <font color='red'>**Reading and testing**</font>

In [ ]:
labels={0: 'CuNi1',
        1: 'CuNi2',
        2: 'CuNi3'}

HEIGHT, WIDTH = 256, 256
batch_size = 8
test_df = pd.read_csv('../data/rgb/test_dry.csv')
test_df.columns = ['path', 'label']
test_df.groupby(['label']).count()

In [ ]:
model = load_model('EfficientNetV2B0dry')

In [ ]:
#Confution Matrix and Classification Report
test_gen = make_generator(test_df, HEIGHT, WIDTH, batch_size)
test_gen.reset()
logits = model.predict(test_gen, test_df.shape[0] // batch_size+1)
y_pred_class = np.argmax(logits, axis=1)

target_names = ['CuNi1', 'CuNi2', 'CuNi3']      

print('Confusion Matrix for experiment: ')#, experiment)
print(confusion_matrix(test_gen.classes, y_pred_class))
print('Classification Report')
print(classification_report(test_gen.classes, y_pred_class, target_names=target_names))

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
target_names = ['CuNi1', 'CuNi2', 'CuNi3']
cm = confusion_matrix(test_gen.classes, y_pred_class, normalize='true')
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=target_names)
disp = disp.plot(include_values=True, cmap=plt.cm.Blues, xticks_rotation='horizontal', values_format='.2f')
plt.show()

In [ ]:
AUC = tf.keras.metrics.AUC()
AUC.update_state(test_gen.classes, y_pred_class)
AUC.result()